In [57]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [58]:
# Shenton Way, District 01, Singapore
address = '16545 sw ,miami'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami home are {}, {}.'.format(latitude, longitude))

C:\Users\Juan Raul Garcia\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Miami home are -34.881491, -56.0571252.


In [43]:
neighborhood_latitude= 25.7467796
neighborhood_longitude= -80.35791434083172

In [59]:
CLIENT_ID = '54GU41KXZPLVE2IUFD2TSPOJVTLCROC1VTWAEYK3YEKIBYU5' # your Foursquare ID
CLIENT_SECRET = 'P2DVHDL0WHSWNSZHSWKJBEBZXRROKWDO2APFCQWQ1CJMFJMG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 54GU41KXZPLVE2IUFD2TSPOJVTLCROC1VTWAEYK3YEKIBYU5
CLIENT_SECRET:P2DVHDL0WHSWNSZHSWKJBEBZXRROKWDO2APFCQWQ1CJMFJMG


In [60]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=54GU41KXZPLVE2IUFD2TSPOJVTLCROC1VTWAEYK3YEKIBYU5&client_secret=P2DVHDL0WHSWNSZHSWKJBEBZXRROKWDO2APFCQWQ1CJMFJMG&v=20180604&ll=48.7467796,2.35791434083172&radius=500&limit=100'

In [61]:
# results display is hidden for report simplification 
results = requests.get(url).json()
#results

In [62]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [63]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(6, 4)

In [64]:
# Venues near current Singapore residence place
SGnearby_venues.head()

,name,categories,lat,lng
0,Class Croute Rungis,Sandwich Place,48.746498,2.356042
1,Brasserie Augusta,French Restaurant,48.745302,2.357272
2,Safran,French Restaurant,48.747531,2.354856
3,Dragon Bleu,Clothing Store,48.743569,2.357812
4,Station Robert Schuman [T7],Tram Station,48.746532,2.352948


In [65]:
latitude=25.7467796
longitude= -80.35791434083172
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)

In [51]:
#add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg